"NousResearch/Nous-Capybara-34B"によるhatespeech_test.csv（600件）のスコアリング

In [1]:
from h2ogpte import H2OGPTE

import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

from tqdm import tqdm

In [2]:
f = open('tmp/h2ogpte_key_hatespeechdetect.txt', 'r')
API_KEY = f.read()
f.close()

In [3]:
client = H2OGPTE(
    address="https://playground.h2ogpte.h2o.ai",
    api_key=API_KEY)
client

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.5.0-dev6`.
You can enable strict version checking by passing strict_version_check=True.


In [4]:
chat_session_id = client.create_chat_session_on_default_collection()

In [5]:
!ls ../tmp

data                     hatespeech_test.csv      hatespeech_train_DAI.csv
hatespeech               hatespeech_test_DAI.csv
hatespeech.zip           hatespeech_train.csv


In [6]:
df = pd.read_csv('../tmp/hatespeech_test.csv')
print(df.shape)
df.head()

(600, 4)


,id,source,text,label
0,e3d3e578b,news4vip,まぁ、俺の言ってることは余りあてにしないでくれwwww\n\n必ずいい人は現れるよ,0
1,a728f26db,news4vip,すまないがそれはレンジのほうか？\nそれともトースター？,0
2,c53e66dc9,livejupiter,そら大不況で独り勝ちした金持ちを殺してしかも自分もちゃんと腹切ってるからな\n今で言うならリ...,0
3,fb3b600fb,newsplus,外国や外国人に税金や公的資金を使うのは違法と言ってくれたら、民進党の支持率は爆上げだろうな。,0
4,ef9cd1326,newsplus,中国人は、日本の年金制度にものらないとw,0


In [7]:
def get_judgements(text_list):
    answers = []
    with client.connect(chat_session_id) as session:
        for text in tqdm(text_list):
            answer = session.query(
                message = text,
                system_prompt=SYS_PROMPT,
                llm=MODEL,
                rag_config={"rag_type": "llm_only",},
            ).content
            answers.append(answer)
    return answers

In [8]:
MODEL = "NousResearch/Nous-Capybara-34B"
SYS_PROMPT = "あなたは文章がヘイトスピーチかどうかを判定するAIです。質問には必ずYes(ヘイトスピーチ)かNo(ヘイトスピーチでない)で答えます。判定の理由は述べる必要はありません。"

outputs = get_judgements(df['text'])

100%|██████████| 600/600 [26:33<00:00,  2.66s/it]


In [9]:
len(outputs)

600

In [10]:
outputs[:20]

['No.',
 'No.',
 'No.',
 'No.',
 'No.',
 'Yes.',
 'No.',
 'No.',
 'No.',
 'No.',
 'Yes.',
 'Yes.',
 'No.',
 'No.',
 'No.',
 'No.',
 'No.',
 'No.',
 'No.',
 'No.']

In [11]:
judges = []
for i, txt in enumerate(outputs):
    firstpart = txt[:15]    # 文字列の先頭
    #print(i, ' : ' ,firstpart)
    judge = 'yes' in firstpart.lower()   # 先頭にYes/yesが含まれるかどうか
    #print('Hate Speech? ', judge)
    judges.append(judge)

In [12]:
df['res'] = judges
df.head()

,id,source,text,label,res
0,e3d3e578b,news4vip,まぁ、俺の言ってることは余りあてにしないでくれwwww\n\n必ずいい人は現れるよ,0,False
1,a728f26db,news4vip,すまないがそれはレンジのほうか？\nそれともトースター？,0,False
2,c53e66dc9,livejupiter,そら大不況で独り勝ちした金持ちを殺してしかも自分もちゃんと腹切ってるからな\n今で言うならリ...,0,False
3,fb3b600fb,newsplus,外国や外国人に税金や公的資金を使うのは違法と言ってくれたら、民進党の支持率は爆上げだろうな。,0,False
4,ef9cd1326,newsplus,中国人は、日本の年金制度にものらないとw,0,False


In [13]:
df['res'].value_counts()

res
False    476
True     124
Name: count, dtype: int64

In [14]:
pd.crosstab(df['label'], df['res'], margins=True)

res,False,True,All
label,,,
0,470,95,565
1,6,29,35
All,476,124,600


In [15]:
f1_score(y_true=df['label'], y_pred=df['res'])

0.36477987421383645

In [16]:
accuracy_score(y_true=df['label'], y_pred=df['res'])

0.8316666666666667

In [17]:
df.to_csv('tmp/hatespeech_test__Nous-Capybara-34B.csv', index=False)